In [ ]:
import logging
import random
from pprint import pformat
from pathlib import Path
import sys

import pandas as pd
import dynamic_yaml
import yaml

sys.path.append("/workspace/correlation-change-predict/utils")
from utils import calc_corr_ser_property

with open('../config/data_config.yaml') as f:
    data = dynamic_yaml.load(f)
    data_cfg = yaml.full_load(dynamic_yaml.dump(data))

logging.basicConfig(level=logging.INFO)

# Prepare data

## selected item setting

In [ ]:
data_implement = "SP500_20112015"  # watch options by printing /config/data_config.yaml/["DATASETS"].keys()
# etl set setting
retrive_items_setting = "-train_all"  # -train_train|-train_all
# set correlation type
corr_type = "pearson"  # "pearson" | "cross_corr"
# set CORR_WINDOW and CORR_STRIDE length
w_l=50 ; s_l = 1
# Decide how to calculate corr_ser
corr_ser_clac_method = "corr_ser_calc_regular"  # corr_ser_calc_regular|corr_ser_calc_abs

## Load Data

In [ ]:
all_set = data_cfg["DATASETS"][data_implement]['ALL_SET']  # all items
train_set = data_cfg["DATASETS"][data_implement]['TRAIN_SET']
items_implement = train_set if retrive_items_setting == "-train_train" else all_set
output_file_name = data_cfg["DATASETS"][data_implement]['OUTPUT_FILE_NAME_BASIS'] + retrive_items_setting
pipeline_corr_data_dir = Path(data_cfg["DIRS"]["PIPELINE_DATA_DIR"])/f"{output_file_name}/{corr_type}"

In [ ]:
dataset_df = pd.read_csv(data_cfg["DATASETS"][data_implement]['FILE_PATH'])
dataset_df = dataset_df.set_index('Date')
dataset_df = dataset_df.loc[::, items_implement]
corr_df = pd.read_csv(pipeline_corr_data_dir/f"corr_data/corr_s{s_l}_w{w_l}.csv", index_col=["item_pair"])
target_df = pd.read_csv(pipeline_corr_data_dir/f"custom_discretize_corr_data/bins_-10_-03_03_10/corr_s{s_l}_w{w_l}.csv", index_col=["item_pair"])
logging.info(f"len(items_implement): {len(items_implement)} and len(all_set): {len(all_set if all_set else [])} and len(train_set): {len(train_set if train_set else [])}")
logging.info(f"dataset_df.shape:{dataset_df.shape}, corr_df.shape:{corr_df.shape}, target_df.shape:{target_df.shape}")
display(dataset_df.head())
display(corr_df.head())
display(target_df.head())

# Random pick trainset

In [ ]:
def gen_random_trainset(all_items: list, train_set_len: int = 100, verbose: int = 0):
    """
    Randon pick items for trainset # Not always necessary to operate
    """
    random.seed(10)
    train_set = sorted(random.sample(all_items, train_set_len))

    if verbose==1:
        logging.info(f"len(train_set):{len(train_set)}")
        pp = pprint.PrettyPrinter(width=500, compact=True)
        pp.pprint(train_set)

    return train_set

print(gen_random_trainset(items_implement, 100))

# Correlation Series Property condition filtered trainset

# ...